# Тестовое задание:

требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.

Для обучения модели используйте данные из файла `train.tsv`. В файле находится таблица, состоящая из двух колонок. 
В колонке title записан заголовок новости. В колонке is_fake содержатся метки: 0 – новость реальная; 1 – новость выдуманная.
Для демонстрации работы модели используйте данные тестового набора из файла `test.tsv`. В нем также есть колонка title, данные которой являются входными для вашей модели.

Вам нужно скопировать файл `test.tsv`, переименовать его в `predictions.tsv` и заполнить колонку is_fake значениями предсказаний вашей модели, аналогично `train.tsv`. 
Изначально колонка заполнена значением 0.

---

In [1]:
# Импорт данных:

import numpy as np
import pandas as pd

train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
#train_data.style.hide_index()
train_data.head(10)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
5,Российским студентам запретят учиться за рубежом,1
6,Путин пишет книгу об истории Украины,1
7,Россияне обхитрили рост цен,0
8,Звезда «Ворониных» раскрыл подробности о своем...,0
9,Microsoft объявила дату выхода очков дополненн...,0


In [2]:
# разделим тренировочный датасет на train/test - чтобы оценить работоспособность модели

from sklearn.model_selection import train_test_split

rs = 13
X_train, X_test, y_train, y_test = train_test_split(train_data['title'], train_data['is_fake'], 
                                                    test_size = 0.1, random_state = rs)
X_train[:10]

5428    Конституционный суд назвал запрет на госидеоло...
1518          Росавиа отложила покупку 65 новых самолетов
1307       Древние люди оказались невосприимчивыми к раку
150     Людмила Путина потребовала половину дворца в Г...
1911    У шахтёров Кузбасса изъяли более 5 тысяч фонар...
2705    Григорий Явлинский открыл отделение партии «Яб...
1493    Явлинский призвал россиян не особенно усердств...
4958    Определился соперник России в финале чемпионат...
3618    Итальянский футболист повторно уличен в употре...
1499    На курортах Краснодарского края отдыхающих буд...
Name: title, dtype: object

# 1. подход TF-IDF:

In [3]:
# Токенизируем входные данные и строим словарь всех известных слов

from sklearn.svm import SVC 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(X_train)

tmp = train_x_vect.toarray().shape
print('по всем {0} сообщениям набралось {1} уникальных слов'.format(tmp[0], tmp[1]))

по всем 5182 сообщениям набралось 17512 уникальных слов


# 1.1. Support Vector Machine:

In [4]:
# Попробуем метод опорных векоторов с линейным ядром - выберем наиболее оптимальный параметр перебором

grid = {'C': np.linspace(0.5, 2, 20)}

cv = KFold(n_splits = 5, shuffle = True, random_state = rs)
clf = SVC(kernel = 'linear', random_state = rs)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.8276066416299332
{'C': 1.6842105263157894}


In [5]:
# лучшая модель:

svc_clf = SVC(kernel = 'linear', random_state = rs, C = gs.best_params_['C'])
svc_clf.fit(train_x_vect, y_train)

SVC(C=1.6842105263157894, kernel='linear', random_state=13)

In [6]:
#20 самых важных для моедли слов - можно также поработать над удалением стоп слов (что, чтобы, как итд)

A = svc_clf.coef_.indices
B = np.abs(svc_clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])

запретят
навального
процентов
рпц
как
млн
лукашенко
чтобы
учёные
шойгу
что
сколково
байден
госдуме
россиян
ленина
нефть
qr
коронавируса
участников


Видно, что у модели есть слова-маркеры вроде короновируса и qr и 2022 - возможно будет лучше использовать word2vec?

In [7]:
#проверяем на тестовой выборке:

from sklearn.metrics import f1_score

y_pred = svc_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

f1_score = 0.8547297297297297


In [8]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = svc_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


Судя по поисковым запросам, есть подозрение, что все эти новости на самом деле правдивые, а само задание просто шутка :)

Либо это знак, что мне надо что-то предпринять для уменьшения колличества ложноположительных ошибок.

---

# 1.2. Random Forest Classifier:

In [9]:
# Найдём оптимальные параметры перебором:

from sklearn.ensemble import RandomForestClassifier as RFC

grid = {'max_depth': np.arange(5, 25)}
cv = KFold(n_splits = 5, shuffle = True, random_state = rs)
clf = RFC(random_state = rs)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.7343389293459712
{'max_depth': 23}


In [10]:

rfc_clf = RFC(max_depth = gs.best_params_['max_depth'], random_state = rs)
rfc_clf.fit(train_x_vect, y_train)

# F1_score на тестовой выборке
y_pred = rfc_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

f1_score = 0.6457023060796646


In [11]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = rfc_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,0
9,Учреждена театральная премия имени Гарольда Пи...,0


Судя по всему модель случайного леса не подходит для данной задачи

---

# 2. Модель с предобработкой:

# 2.1 StopWords deliting + TF-IDF + SVC:

In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Импорт даных + удаление стоп слов:
train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
stop_words = set(stopwords.words('russian'))

for i in range(len(train_data)):
    
    words = word_tokenize(train_data['title'][i])
    wordsFiltered = []

    for w in words:
        if w not in stop_words:
            wordsFiltered.append(w)
            
    train_data['title'][i] = " ".join(wordsFiltered)
    
# Разделение на тестовую и тренировочную выборку:
X_train, X_test, y_train, y_test = train_test_split(train_data['title'], train_data['is_fake'], 
                                                    test_size = 0.1, random_state = rs)

# TF-IDF
vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(X_train)

# подбор лучших параметров для модели:
grid = {'C': np.linspace(0.5, 2, 20)}

cv = KFold(n_splits = 5, shuffle = True, random_state = rs)
clf = SVC(kernel = 'linear', random_state = rs)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)

# обучение лучшей модели:
svc_sw_clf = SVC(kernel = 'linear', random_state = rs, C = gs.best_params_['C'])
svc_sw_clf.fit(train_x_vect, y_train)

# Проверка метрикой F1:
y_pred = svc_sw_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

#Точность от удаления стоп слов не то чтобы увеличилась

<ipython-input-12-59a93fe081a4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['title'][i] = " ".join(wordsFiltered)


f1_score = 0.8344827586206898


In [13]:
# 20 наиболее значимых для модели слов ..

A = svc_sw_clf.coef_.indices
B = np.abs(svc_sw_clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])

запретят
навального
рпц
процентов
лукашенко
млн
сколково
шойгу
байден
учёные
ленина
участников
госдуме
коронавируса
россиян
граждан
qr
twitter
2022
роскосмос


In [14]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = svc_sw_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


# 2.2 StopWords deliting + Lemmatization + TF-IDF + SVC:

In [15]:
import pymorphy2

# В ходе исслаодвания также пробовал стеммер, но он не всегда верно обрабатывал слова (Навальный -> Навальн итд)
#from nltk.stem.snowball import SnowballStemmer
#stemmer = SnowballStemmer("russian")

# Импорт даных:
train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
morph = pymorphy2.MorphAnalyzer()

    
# удаление стоп слов:
for i in range(len(train_data)):
    
    words = word_tokenize(train_data['title'][i])
    wordsFiltered = []

    for w in words:
        if w not in stop_words:
            wordsFiltered.append(w)
            
    train_data['title'][i] = " ".join(wordsFiltered)
    
    
# лемантизация (# = стеммер):
for i in range(len(train_data)):
    
    title_words = word_tokenize(train_data['title'][i])
    lemmatized_title = []
    #stemmed_title = []

    for word in title_words:
        
        p = morph.parse(word)[0]
        lemmatized_title.append(p.normal_form)
        #stemmed_title.append(stemmer.stem(word))

    train_data['title'][i] = " ".join(lemmatized_title)
    #train_data['title'][i] = " ".join(stemmed_title)
    
    
# Разделение на тестовую и тренировочную выборку:
rs = 777
X_train, X_test, y_train, y_test = train_test_split(train_data['title'], train_data['is_fake'], 
                                                    test_size = 0.1, random_state = rs)


# TF-IDF
vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(X_train)


# подбор лучших параметров для модели:
grid = {'C': np.linspace(0.5, 2, 20)}

cv = KFold(n_splits = 5, shuffle = True, random_state = rs)
clf = SVC(kernel = 'linear', random_state = rs)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)


# лучшая модель:
best_clf = SVC(kernel = 'linear', random_state = rs, C = gs.best_params_['C'])
best_clf.fit(train_x_vect, y_train)


# Проверка метрикой F1:
y_pred = best_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

<ipython-input-15-49070151d08f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['title'][i] = " ".join(wordsFiltered)
<ipython-input-15-49070151d08f>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['title'][i] = " ".join(lemmatized_title)


f1_score = 0.8547579298831386


In [17]:
# 20 наиболее значимых для модели слов:

A = best_clf.coef_.indices
B = np.abs(best_clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])

навальный
гражданин
байден
запретить
млн
который
депутат
рпц
вакцина
обязать
житель
лукашенко
дмитрий
миллиард
выборы
канал
футболист
сколково
россиянин
право


In [18]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = best_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1
4,Акции Google рухнули после объявления о переза...,1
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,0
9,Учреждена театральная премия имени Гарольда Пи...,0


---

# 3.Вывод:

Наилучшую точность по метрике F1 показал подход TF_IDF  в сочетании с SVC 

In [19]:
test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = best_clf.predict(vectorizer.transform(test_data['title']))
test_data.to_csv('predictions.tsv', index = False, sep = '\t')